In [1]:
%pip install --upgrade --quiet  langchain langchain-community langchainhub gpt4all langchain-chroma sentence_transformers grounded-ai

In [2]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
data = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
all_splits = text_splitter.split_documents(data)

In [3]:
from langchain_chroma import Chroma
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

model_name = "nomic-ai/nomic-embed-text-v1"
model_kwargs = {
    'device': 'cuda',
    'trust_remote_code':True
    }
encode_kwargs = {'normalize_embeddings': True}
hf = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs,
    query_instruction = "search_query:",
    embed_instruction = "search_document:"
)

vectorstore = Chroma.from_documents(documents=all_splits, embedding=hf)

tokenizer_config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

In [4]:
question = "What are the approaches to Task Decomposition?"
docs = vectorstore.similarity_search(question)
len(docs)

4

In [5]:
docs[0].page_content

'Task decomposition can be done (1) by LLM with simple prompting like "Steps for XYZ.\\n1.", "What are the subgoals for achieving XYZ?", (2) by using task-specific instructions; e.g. "Write a story outline." for writing a novel, or (3) with human inputs.'

In [6]:
custom_prompt = """
        You are comparing a reference text to a question and trying to determine if the reference text
        contains ANY information relevant to answering the question. Focus on if the reference has the answer anywhere in the text.
        Here is the data:
        [BEGIN DATA]
        ************
        [Question]: {{ query }}
        ************
        [Reference text]: {{ text }}
        ************
        [END DATA]
        Compare the Question above to the Reference text. You must determine whether the Reference text
        contains information that can answer the Question. Please focus on whether the very specific
        question can be answered by the information in the Reference text.
        Your response must be single word, either "relevant" or "unrelated",
        and should not contain any text or characters aside from that word.
        "unrelated" means that the reference text does not contain an answer to the Question.
        "relevant" means the reference text contains an answer to the Question.
        """

In [7]:
from grounded_ai.evaluators.rag_relevance_evaluator import RagRelevanceEvaluator
rag_relevance_evaluator = RagRelevanceEvaluator(base_prompt=custom_prompt)
rag_relevance_evaluator.warmup()

You are attempting to use Flash Attention 2.0 with a model not initialized on GPU. Make sure to move the model to GPU after initializing it on CPU with `model.to('cuda')`.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/686 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/25.2M [00:00<?, ?B/s]

In [8]:
data = []
for doc in docs:
  pair = [question, doc.page_content]
  data.append(pair)
data

[['What are the approaches to Task Decomposition?',
  'Task decomposition can be done (1) by LLM with simple prompting like "Steps for XYZ.\\n1.", "What are the subgoals for achieving XYZ?", (2) by using task-specific instructions; e.g. "Write a story outline." for writing a novel, or (3) with human inputs.'],
 ['What are the approaches to Task Decomposition?',
  'Planning\n\nSubgoal and decomposition: The agent breaks down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.\nReflection and refinement: The agent can do self-criticism and self-reflection over past actions, learn from mistakes and refine them for future steps, thereby improving the quality of final results.\n\n\nMemory'],
 ['What are the approaches to Task Decomposition?',
  'Challenges in long-term planning and task decomposition: Planning over a lengthy history and effectively exploring the solution space remain challenging. LLMs struggle to adjust plans when faced with unexpect

In [11]:
for doc in docs:
  print(doc.page_content)

Task decomposition can be done (1) by LLM with simple prompting like "Steps for XYZ.\n1.", "What are the subgoals for achieving XYZ?", (2) by using task-specific instructions; e.g. "Write a story outline." for writing a novel, or (3) with human inputs.
Planning

Subgoal and decomposition: The agent breaks down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.
Reflection and refinement: The agent can do self-criticism and self-reflection over past actions, learn from mistakes and refine them for future steps, thereby improving the quality of final results.


Memory
Challenges in long-term planning and task decomposition: Planning over a lengthy history and effectively exploring the solution space remain challenging. LLMs struggle to adjust plans when faced with unexpected errors, making them less robust compared to humans who learn from trial and error.
Fig. 1. Overview of a LLM-powered autonomous agent system.
Component One: Planning#
A compli

In [10]:
response = rag_relevance_evaluator.evaluate(data)
response

{'relevant': 1, 'unrelated': 3, 'percentage_relevant': 25.0}